In [1]:
from functools import partial
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import mlflow
import mlflow.lightgbm
import lightgbm as lgb

from utils import HPOpt

# Load data

In [2]:
iris_data = load_iris()

col_names = ['_'.join(feat.split(' ')[:2]) for feat in iris_data['feature_names']]
X, y = pd.DataFrame(iris_data['data'],columns=col_names),pd.Series(iris_data['target'])
X_test, X_train, y_test, y_train = train_test_split(X, y, train_size=.8, random_state=42, stratify=y)


In [3]:
print(X.shape)
X.head()

(150, 4)


,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


# Defining model parameters and training parameters

In [4]:
bst_params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'min_child_samples':3,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'seed': 42,
    'num_boost_round': 30,
    'verbose_eval': 5,
    'early_stopping_rounds': 5,
}


In [5]:
classifier_params_space = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, .31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(1, 4, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'min_child_samples':hp.choice('min_child_samples', np.arange(1, 10, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'num_boost_round':  hp.choice('num_boost_round', np.arange(5, 20, 1)),
}
fit_params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    #'eval_metric': 'logloss',
    'early_stopping_rounds': 3,
    'verbose_eval': False,
}
lgb_para = dict()
lgb_para['params_space'] = classifier_params_space
lgb_para['fit_params'] = fit_params
lgb_para['scoring_func'] = partial(f1_score, average='micro')



# First experiment 

In [6]:
obj = HPOpt(X_train, X_test, y_train, y_test, 'experiment 3')

hp_results = obj.process(space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=20)

[1]	train's multi_logloss: 0.962841	valid's multi_logloss: 0.988288
Training until validation scores don't improve for 3 rounds
[2]	train's multi_logloss: 0.907459	valid's multi_logloss: 0.927698
[3]	train's multi_logloss: 0.907459	valid's multi_logloss: 0.927698
[4]	train's multi_logloss: 0.868594	valid's multi_logloss: 0.879207
[5]	train's multi_logloss: 0.868594	valid's multi_logloss: 0.879207
[6]	train's multi_logloss: 0.866299	valid's multi_logloss: 0.878272
[7]	train's multi_logloss: 0.866299	valid's multi_logloss: 0.878272
[8]	train's multi_logloss: 0.864651	valid's multi_logloss: 0.877766
[9]	train's multi_logloss: 0.863806	valid's multi_logloss: 0.875545
[10]	train's multi_logloss: 0.862357	valid's multi_logloss: 0.875187
Did not meet early stopping. Best iteration is:     
[10]	train's multi_logloss: 0.862357	valid's multi_logloss: 0.875187
[1]	train's multi_logloss: 1.00436	valid's multi_logloss: 1.02121           
Training until validation scores don't improve for 3 rounds 

/anaconda2/envs/py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

/anaconda2/envs/py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[3]	train's multi_logloss: 0.924048	valid's multi_logloss: 0.946135         
[4]	train's multi_logloss: 0.891053	valid's multi_logloss: 0.910083         
[5]	train's multi_logloss: 0.890644	valid's multi_logloss: 0.908851         
[6]	train's multi_logloss: 0.865053	valid's multi_logloss: 0.877185         
[7]	train's multi_logloss: 0.865053	valid's multi_logloss: 0.877185         
[8]	train's multi_logloss: 0.843257	valid's multi_logloss: 0.849853         
Did not meet early stopping. Best iteration is:                             
[8]	train's multi_logloss: 0.843257	valid's multi_logloss: 0.849853
[1]	train's multi_logloss: 0.981833	valid's multi_logloss: 1.01194           
Training until validation scores don't improve for 3 rounds                  
[2]	train's multi_logloss: 0.839649	valid's multi_logloss: 0.884629          
[3]	train's multi_logloss: 0.765518	valid's multi_logloss: 0.821402          
[4]	train's multi_logloss: 0.672852	valid's multi_logloss: 0.737885          
[5]

Early stopping, best iteration is:                                           
[1]	train's multi_logloss: 1.09861	valid's multi_logloss: 1.09861
[1]	train's multi_logloss: 0.997469	valid's multi_logloss: 1.01271           
Training until validation scores don't improve for 3 rounds                  
[2]	train's multi_logloss: 0.86639	valid's multi_logloss: 0.875495           
[3]	train's multi_logloss: 0.812446	valid's multi_logloss: 0.829446          
[4]	train's multi_logloss: 0.722748	valid's multi_logloss: 0.734347          
[5]	train's multi_logloss: 0.697607	valid's multi_logloss: 0.708369          
Did not meet early stopping. Best iteration is:                              
[5]	train's multi_logloss: 0.697607	valid's multi_logloss: 0.708369
[1]	train's multi_logloss: 0.871424	valid's multi_logloss: 0.914253           
Training until validation scores don't improve for 3 rounds                   
[2]	train's multi_logloss: 0.715807	valid's multi_logloss: 0.791476           
[3]	t

[5]	train's multi_logloss: 0.402298	valid's multi_logloss: 0.517667            
[6]	train's multi_logloss: 0.349721	valid's multi_logloss: 0.453541            
[7]	train's multi_logloss: 0.330386	valid's multi_logloss: 0.438539            
[8]	train's multi_logloss: 0.292454	valid's multi_logloss: 0.393917            
[9]	train's multi_logloss: 0.27971	valid's multi_logloss: 0.384783             
[10]	train's multi_logloss: 0.267117	valid's multi_logloss: 0.366139           
[11]	train's multi_logloss: 0.266952	valid's multi_logloss: 0.366595           
[12]	train's multi_logloss: 0.2647	valid's multi_logloss: 0.365909             
[13]	train's multi_logloss: 0.264508	valid's multi_logloss: 0.366251           
[14]	train's multi_logloss: 0.262812	valid's multi_logloss: 0.365815           
[15]	train's multi_logloss: 0.259021	valid's multi_logloss: 0.364441           
[16]	train's multi_logloss: 0.257464	valid's multi_logloss: 0.363971           
[17]	train's multi_logloss: 0.257344	val

# Second experiment 
Lets add some other features

In [7]:
X['sepal_area'] = X.sepal_length*X.sepal_width
X['petal_area'] = X.petal_length*X.petal_width
X_test, X_train, y_test, y_train = train_test_split(X, y, train_size=.8, random_state=42, stratify=y)


In [8]:
obj = HPOpt(X_train, X_test, y_train, y_test, 'experiment 2')

hp_results = obj.process(space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=20)

[1]	train's multi_logloss: 0.998215	valid's multi_logloss: 1.00451
Training until validation scores don't improve for 3 rounds
[2]	train's multi_logloss: 0.922704	valid's multi_logloss: 0.932428
[3]	train's multi_logloss: 0.887907	valid's multi_logloss: 0.893875
[4]	train's multi_logloss: 0.865227	valid's multi_logloss: 0.86549
[5]	train's multi_logloss: 0.863992	valid's multi_logloss: 0.865088
[6]	train's multi_logloss: 0.863789	valid's multi_logloss: 0.865576
[7]	train's multi_logloss: 0.862862	valid's multi_logloss: 0.865364
[8]	train's multi_logloss: 0.862113	valid's multi_logloss: 0.865255
Early stopping, best iteration is:                  
[5]	train's multi_logloss: 0.863992	valid's multi_logloss: 0.865088
[1]	train's multi_logloss: 0.914672	valid's multi_logloss: 0.936293          
Training until validation scores don't improve for 3 rounds                  
[2]	train's multi_logloss: 0.772934	valid's multi_logloss: 0.804268          
[3]	train's multi_logloss: 0.661408	valid's

/anaconda2/envs/py36/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

/anaconda2/envs/py36/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[6]	train's multi_logloss: 0.43055	valid's multi_logloss: 0.499923           
[7]	train's multi_logloss: 0.377721	valid's multi_logloss: 0.4569            
[8]	train's multi_logloss: 0.329466	valid's multi_logloss: 0.407362          
[9]	train's multi_logloss: 0.291117	valid's multi_logloss: 0.364938          
[10]	train's multi_logloss: 0.25851	valid's multi_logloss: 0.337632          
[11]	train's multi_logloss: 0.228871	valid's multi_logloss: 0.307098         
[12]	train's multi_logloss: 0.203802	valid's multi_logloss: 0.283281         
[13]	train's multi_logloss: 0.182184	valid's multi_logloss: 0.261674         
[14]	train's multi_logloss: 0.163815	valid's multi_logloss: 0.24846          
[15]	train's multi_logloss: 0.149694	valid's multi_logloss: 0.241573         
[16]	train's multi_logloss: 0.133941	valid's multi_logloss: 0.225313         
[17]	train's multi_logloss: 0.123914	valid's multi_logloss: 0.218179         
Did not meet early stopping. Best iteration is:                 

[1]	train's multi_logloss: 1.0374	valid's multi_logloss: 1.03724              
Training until validation scores don't improve for 3 rounds                   
[2]	train's multi_logloss: 0.964901	valid's multi_logloss: 0.963426           
[3]	train's multi_logloss: 0.900319	valid's multi_logloss: 0.896657           
[4]	train's multi_logloss: 0.860462	valid's multi_logloss: 0.850107           
[5]	train's multi_logloss: 0.832513	valid's multi_logloss: 0.821543           
[6]	train's multi_logloss: 0.792317	valid's multi_logloss: 0.781382           
[7]	train's multi_logloss: 0.752558	valid's multi_logloss: 0.744261           
Did not meet early stopping. Best iteration is:                               
[7]	train's multi_logloss: 0.752558	valid's multi_logloss: 0.744261
[1]	train's multi_logloss: 1.09861	valid's multi_logloss: 1.09861              
Training until validation scores don't improve for 3 rounds                    
[2]	train's multi_logloss: 1.09861	valid's multi_logloss: 1.0

[2]	train's multi_logloss: 0.725127	valid's multi_logloss: 0.770678            
[3]	train's multi_logloss: 0.593079	valid's multi_logloss: 0.635748            
[4]	train's multi_logloss: 0.505755	valid's multi_logloss: 0.562343            
[5]	train's multi_logloss: 0.451845	valid's multi_logloss: 0.511759            
[6]	train's multi_logloss: 0.381803	valid's multi_logloss: 0.447061            
[7]	train's multi_logloss: 0.332409	valid's multi_logloss: 0.404249            
[8]	train's multi_logloss: 0.293758	valid's multi_logloss: 0.359638            
[9]	train's multi_logloss: 0.27702	valid's multi_logloss: 0.352926             
[10]	train's multi_logloss: 0.25093	valid's multi_logloss: 0.3276              
[11]	train's multi_logloss: 0.227512	valid's multi_logloss: 0.305335           
[12]	train's multi_logloss: 0.209124	valid's multi_logloss: 0.28641            
[13]	train's multi_logloss: 0.201529	valid's multi_logloss: 0.270999           
[14]	train's multi_logloss: 0.199345	val